In [1]:
import os
import pandas as pd
from exg.ema import EMA
from exg.sma import SMA
import json

In [2]:
#  Load the JSON data from the file
def load_finger_thresholds():
    with open('../config/finger_thresholds.json', 'r') as file:
        data = json.load(file)

    # Convert lists back to tuples if necessary
    finger_thresholds = tuple(tuple(pair) for pair in data['finger_thresholds'])

    return finger_thresholds

finger_thresholds = load_finger_thresholds()

In [11]:
# Load the data from the data csv file
def load_data(session_folder, csv_num, raw_columns, finger_columns):
    """Loads the data from a given data csv file."""
    # Load the data with timestamp
    exg_data = pd.read_csv(session_folder + f"/exg/data_{csv_num}.csv")
    angle_data = pd.read_csv(session_folder + f"/angle/data_{csv_num}.csv")
    
    # Select the columns we need
    exg_data = exg_data[raw_columns + ["timestamp"]]
    angle_data = angle_data[finger_columns + ["timestamp"]]
    
    # Convert the timestamp to a timedelta
    exg_data['timestamp'] =  pd.to_timedelta(exg_data["timestamp"], unit='ms')
    angle_data['timestamp'] =  pd.to_timedelta(angle_data["timestamp"], unit='ms')
    
    # Calculate sample rate based on size of the data and duration
    exg_duration = (exg_data["timestamp"].iloc[-1] - exg_data["timestamp"].iloc[0]).total_seconds()
    angle_duration = (angle_data["timestamp"].iloc[-1] - angle_data["timestamp"].iloc[0]).total_seconds()
    exg_sample_rate = round(len(exg_data) / exg_duration)
    angle_sample_rate = round(len(angle_data) / angle_duration)
    
    # Process the exg data
    window_sizes = [2, 4, 8, 16, 32, 64, 128]  # Rolling window sizes in number of frames
    num_channels = len(raw_columns)  # Number of channels

    # Initialize the rolling processor
    ema_processor = EMA(
        window_sizes=window_sizes,
        num_channels=num_channels,
        fs=exg_sample_rate
    )
    sma_processor = SMA(
        window_sizes=window_sizes,
        num_channels=num_channels,
        fs=exg_sample_rate
    )
    ema_data = ema_processor.process(exg_data[raw_columns].values)
    ema_data = ema_processor.results_to_df(ema_data)
    ema_columns = ema_data.columns
    
    sma_data = sma_processor.process(exg_data[raw_columns].values)
    sma_data = sma_processor.results_to_df(sma_data)
    sma_columns = sma_data.columns
    
    # Merge the processed data with the original data
    processed_exg = pd.concat([exg_data, ema_data, sma_data], axis=1)
    
    # Calculate the tolerance for merging the data
    tolerance = round(1 / max(exg_sample_rate, angle_sample_rate) * 1000)
    
    # Merge the data
    data = pd.merge_asof(processed_exg, angle_data, on="timestamp", direction='nearest', tolerance=pd.Timedelta(tolerance, unit='ms'))
    data = data.set_index("timestamp")
    
    # Remove rows with NaN values
    data = data.dropna()
    
    return data, ema_columns, sma_columns, sma_processor.window_intervals_ms

def load_session(session_folder, raw_columns, finger_columns):
    """
    Loads all csv files in a given data folder.
    
    Parameters:
    session_folder (str): The folder containing the data.
    raw_columns (list): The columns to load from the raw data.
    finger_columns (list): The columns to load from the finger data.
    
    Returns:
    session (list): A list of dataframes containing the data.
    """
    # Get all csv files in the folder
    csv_files = [f for f in os.listdir(session_folder + "/exg") if f.endswith('.csv')]
    
    # Load all csv files
    session = []
    ema_columns = None
    sma_columns = None
    window_intervals_ms = None
    for csv in csv_files:
        csv_num = int(csv.split("_")[1].split(".")[0])
        print(f"Loaded data_{csv_num}.csv")
        data, ema_columns, sma_columns, window_intervals_ms = load_data(session_folder, csv_num, raw_columns, finger_columns)
        session.append(data)
    
    return session, ema_columns, sma_columns, window_intervals_ms

# Define the columns for the data
exg_channels = [1, 2, 3, 4, 5, 6, 7, 8]
finger_columns = ["thumb", "index", "middle", "pinky"]
raw_columns = []
for emg_channel in exg_channels:
    raw_columns.append(f"ch_{emg_channel}_raw")

# Load all sessions in the data/ directory
session_folder = "../data/s_2"
session, ema_columns, sma_columns, window_intervals_ms = load_session(session_folder, raw_columns, finger_columns)

Loaded data_1734921686.csv
Loaded data_1734921116.csv


In [12]:
session[0][raw_columns]

,ch_1_raw,ch_2_raw,ch_3_raw,ch_4_raw,ch_5_raw,ch_6_raw,ch_7_raw,ch_8_raw
timestamp,,,,,,,,
0 days 00:00:00.033000,-4.211684,7.024826,85.023705,9.719947,33.806087,14.691137,33.542664,49.870438
0 days 00:00:00.037000,-40.660114,11.993863,56.987484,21.194567,12.367243,-20.725056,27.311964,26.786173
0 days 00:00:00.041000,-30.935337,3.830864,-6.577931,16.492908,10.467344,-60.642273,12.136725,-21.833904
0 days 00:00:00.045000,6.837028,-4.851535,-79.330590,-1.841237,-13.111038,-34.049680,33.303390,-15.896152
0 days 00:00:00.049000,-46.247585,-6.937012,-68.733986,4.368188,-45.338170,22.131365,20.994452,-30.093897
...,...,...,...,...,...,...,...,...
0 days 00:04:43.708000,-35.501583,-2.214828,-1.097068,-1.985924,-4.237298,-3.966445,5.390298,-0.667433
0 days 00:04:43.712000,-2.318553,-0.570094,-1.771064,2.501905,-5.552795,-11.954399,12.212945,1.295303
0 days 00:04:43.716000,-16.685910,-0.062907,-1.158734,10.322027,-3.802270,-3.794136,7.889131,-0.162088


In [13]:
session[0][sma_columns]

,ch_1_sma_mean_8ms,ch_2_sma_mean_8ms,ch_3_sma_mean_8ms,ch_4_sma_mean_8ms,ch_5_sma_mean_8ms,ch_6_sma_mean_8ms,ch_7_sma_mean_8ms,ch_8_sma_mean_8ms,ch_1_sma_mean_16ms,ch_2_sma_mean_16ms,...,ch_7_sma_variance_256ms,ch_8_sma_variance_256ms,ch_1_sma_variance_512ms,ch_2_sma_variance_512ms,ch_3_sma_variance_512ms,ch_4_sma_variance_512ms,ch_5_sma_variance_512ms,ch_6_sma_variance_512ms,ch_7_sma_variance_512ms,ch_8_sma_variance_512ms
timestamp,,,,,,,,,,,,,,,,,,,,,
0 days 00:00:00.033000,2.105842,3.512413,42.511853,4.859973,16.903043,7.345568,16.771332,24.935219,1.052921,1.756207,...,17.305163,38.253129,0.137498,0.382521,56.035575,0.732338,8.858773,1.672995,8.721253,19.278363
0 days 00:00:00.037000,22.435899,9.509345,71.005595,15.457257,23.086665,17.708096,30.427314,38.328305,11.217949,4.754672,...,28.331091,48.636617,12.931663,1.487305,80.617560,4.189223,9.993314,4.965298,14.391576,24.676965
0 days 00:00:00.041000,35.797725,7.912363,31.782708,18.843737,11.417294,40.683665,19.724345,24.310038,18.951784,5.712388,...,30.236061,55.151727,20.180328,1.592168,80.838929,6.235506,10.783607,33.209023,15.443210,28.167928
0 days 00:00:00.045000,18.886183,4.341199,42.954261,9.167072,11.789191,47.345977,22.720058,18.865028,20.661041,6.925272,...,46.108271,58.273818,20.479401,1.761085,128.182628,6.251129,12.025427,41.796660,23.743746,29.935512
0 days 00:00:00.049000,26.542307,5.894274,74.032288,3.104712,29.224604,28.090523,27.148921,22.995025,31.170016,6.903318,...,51.797999,70.522549,36.591973,2.110645,162.891012,6.372775,27.572911,45.241808,26.887924,36.535361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 00:04:43.708000,41.113718,2.224610,2.037492,1.638359,2.856195,3.315072,9.960259,1.880954,23.698321,1.247645,...,58.219041,0.958873,335.108121,1.340839,2.150644,19.907357,2.359332,7.739075,53.317393,1.063441
0 days 00:04:43.712000,18.910068,1.392461,1.434066,2.243915,4.895046,7.960422,8.801622,0.981368,22.749089,1.289159,...,57.681771,0.930450,337.051001,1.343843,2.130607,19.966055,2.439257,8.229846,53.300461,1.063443
0 days 00:04:43.716000,9.502232,0.316501,1.464899,6.411966,4.677532,7.874268,10.051038,0.728696,25.307975,1.270555,...,57.513725,0.937551,335.763077,1.343756,1.916068,19.964911,2.451893,8.225941,53.242977,1.074198


In [10]:
session[0][ema_columns]

,ch_1_ema_mean_8ms,ch_2_ema_mean_8ms,ch_3_ema_mean_8ms,ch_4_ema_mean_8ms,ch_5_ema_mean_8ms,ch_6_ema_mean_8ms,ch_7_ema_mean_8ms,ch_8_ema_mean_8ms,ch_1_ema_mean_16ms,ch_2_ema_mean_16ms,...,ch_7_ema_variance_256ms,ch_8_ema_variance_256ms,ch_1_ema_variance_512ms,ch_2_ema_variance_512ms,ch_3_ema_variance_512ms,ch_4_ema_variance_512ms,ch_5_ema_variance_512ms,ch_6_ema_variance_512ms,ch_7_ema_variance_512ms,ch_8_ema_variance_512ms
timestamp,,,,,,,,,,,,,,,,,,,,,
0 days 00:00:00.034000,2.807789,4.683218,56.682470,6.479965,22.537391,9.794091,22.361776,33.246959,1.684674,2.809931,...,33.553586,74.170328,0.270748,0.753226,110.340348,1.442056,17.443921,3.294315,17.173128,37.961264
0 days 00:00:00.042000,29.971115,5.739570,23.347225,16.425172,12.230660,46.121982,16.645117,24.202525,22.739045,5.422448,...,54.644580,99.473490,38.984213,3.043653,153.809806,11.967502,20.468723,64.632235,29.346245,53.474984
0 days 00:00:00.049000,35.681187,6.340524,66.045813,5.146308,34.497973,27.445504,23.246512,26.284246,28.325977,5.891254,...,88.905838,119.710470,69.171710,3.903469,303.423601,11.818648,52.034481,84.783340,49.754702,67.233717
0 days 00:00:00.059000,6.668929,10.258236,44.124642,4.733669,7.040360,12.558371,50.059745,13.165999,12.209723,8.437681,...,214.532725,134.410937,67.098524,6.317636,331.536698,11.916522,50.519230,83.880057,118.881489,78.345895
0 days 00:00:00.067000,20.346153,7.744626,72.466047,12.834999,19.863474,20.522051,26.045216,16.623251,20.243058,7.186748,...,317.770366,237.681900,87.869116,7.247775,472.765244,39.977805,67.447725,136.069957,181.145865,137.775837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 00:04:43.691000,19.212197,0.825566,1.871697,5.006221,2.264851,7.189777,24.334834,1.540287,19.267162,0.910220,...,67.957848,0.963948,303.898152,1.319392,2.104995,23.598814,2.706385,9.533081,61.868292,1.121322
0 days 00:04:43.699000,7.793929,0.273064,0.692964,4.146293,3.206118,8.772112,14.462498,0.945702,10.984041,0.479577,...,64.424290,0.922636,302.252351,1.316174,2.086608,23.401742,2.663795,10.072772,60.327513,1.096604
0 days 00:04:43.707000,34.917237,2.003424,1.470134,2.071492,3.508899,4.210909,8.429414,1.237694,29.369093,1.594833,...,61.787280,0.971949,304.923673,1.298658,2.050311,23.246337,2.648345,9.794372,59.117957,1.115623
